# Dataset Conversion: MATLAB to JSON

This notebook converts battery test data from MATLAB `.mat` files to a unified JSON format for easier processing and analysis.

## Purpose
- Load battery measurement data from MATLAB files in the `CU_Dynamic` directory
- Extract data for battery cell BW-VTC-458 from each checkup (CU000, CU001, etc.)
- Convert numpy arrays and data types to JSON-serializable format
- Save the complete dataset as `full_data.json`

## Output Structure
The resulting JSON file contains:
```
{
  "CU000": {
    "Time": [...],
    "I": [...],        // Current measurements
    "U": [...],        // Voltage measurements
    "Ah": [...],       // Capacity measurements
    "Command": [...]   // Test commands (Charge, Discharge, Pause)
  },
  "CU001": { ... },
  ...
}
```

In [ ]:
# Import required libraries for data processing
from scipy.io import loadmat  # For loading MATLAB .mat files
from os import listdir  # For listing directory contents
from json import dump  # For saving data to JSON format
import numpy as np  # For handling numpy arrays and data types

## Import Required Libraries
Setting up the necessary tools for reading MATLAB files, file system operations, and JSON serialization.

In [ ]:
def make_json_serializable(obj):
    """
    Recursively convert numpy objects to JSON-serializable Python types.
    This function handles various numpy data types that cannot be directly serialized to JSON.
    """
    # Handle dictionaries recursively
    if isinstance(obj, dict):
        return {k: make_json_serializable(v) for k, v in obj.items()}
    
    # Convert numpy arrays to lists and process recursively
    if isinstance(obj, np.ndarray):
        return make_json_serializable(obj.tolist())
    
    # Handle lists and tuples recursively
    if isinstance(obj, (list, tuple)):
        return [make_json_serializable(x) for x in obj]
    
    # Convert numpy integer types to Python int
    if isinstance(obj, (np.integer,)):
        return int(obj)
    
    # Convert numpy floating point types to Python float
    if isinstance(obj, (np.floating,)):
        return float(obj)
    
    # Convert numpy boolean to Python bool
    if isinstance(obj, np.bool_):
        return bool(obj)
    
    # Handle other numpy generic types
    if isinstance(obj, np.generic):
        return obj.item()
    
    # Return object as-is if already JSON serializable
    return obj

# Dictionary to store all processed data organized by checkup
full_dict = {}

# Process each checkup directory in sorted order
for checkup_name in sorted(listdir("CU_Dynamic")):
    # Skip the failed/incomplete data directory
    if checkup_name == "failed and incomplete":
        continue
    
    # Initialize dictionary for this checkup
    full_dict[checkup_name] = {}
    
    # Process each file in the checkup directory
    for cell in listdir(f"CU_Dynamic/{checkup_name}"):
        # Only process files for battery cell BW-VTC-458
        if cell.startswith("BW-VTC-458"):
            # Load MATLAB file and extract Dataset
            data = loadmat(
                f"CU_Dynamic/{checkup_name}/{cell}",
                squeeze_me=True,        # Remove singleton dimensions
                struct_as_record=False  # Return structs as objects, not record arrays
            )["Dataset"].__dict__
            
            # Copy all relevant data fields to our dictionary
            for key in data.keys():
                # Skip metadata fields that aren't needed
                if key in ["_fieldnames", "Info"]:
                    continue
                full_dict[checkup_name][key] = data[key]
            
            # Only process the first matching file per checkup
            break

# Save the complete dataset to JSON file
with open("full_data.json", "w") as f:
    dump(make_json_serializable(full_dict), f)

## Data Processing and Conversion
This section handles the conversion of MATLAB data to JSON format, including:
1. **JSON Serialization Function**: Converts numpy arrays and data types to JSON-compatible Python types
2. **Data Extraction**: Loads battery data from each checkup directory 
3. **File Processing**: Processes only BW-VTC-458 battery cell data from each checkup
4. **JSON Export**: Saves the unified dataset to `full_data.json`

## Conversion Complete ✅

The dataset conversion process is now complete. The `full_data.json` file contains:

- **Battery cell**: BW-VTC-458 measurements
- **Time series data**: Current (I), Voltage (U), Capacity (Ah), and Commands
- **Checkups**: All available checkup cycles (CU000, CU001, CU002, etc.)
- **Format**: JSON-serialized data ready for analysis

### Next Steps
The generated `full_data.json` file can now be used for:
- Battery state analysis
- Degradation studies  
- Machine learning applications
- Data visualization
- Statistical analysis

### File Location
📁 `full_data.json` - Generated in the current working directory